In [2]:
import requests
from requests import Response
from pydantic import BaseModel,RootModel,Field,field_validator
import pandas as pd

def get_urldata() -> Response | None:
    json_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
    try:
        responseData:Response = requests.get(json_url)
        responseData.raise_for_status()
        if responseData.status_code == 200:
            print("下載成功")
            return responseData
        else:
            print("下載失敗")
            return None
    except Exception as a:
        print(a)
        print("連線失敗")
        return None
##################################
class Site(BaseModel):
    站點名稱:str = Field(alias='sna')
    行政區:str = Field(alias='sarea')
    總車輛數:float = Field(alias='tot')
    可借:float = Field(alias='sbi')
    可還:float = Field(alias='bemp')
    時間:str = Field(alias='mday')
    
    @field_validator("總車輛數","可借","可還",mode='before')
    @classmethod
    def empty_to_zero(cls, value):
        if value == '':
            return '0.0'
        else:
            return value

class Sites(RootModel):
    root:list[Site]
    def __iter__(self):
        return iter(self.root)
    def __getitem__(self, item):
        return self.root(item)
##################################
#使用def main方式會無法顯示最終答案表格
#def main():
rawData:Response | None = get_urldata()
rawstr:str = rawData.text
Datas:Sites = Sites.model_validate_json(rawstr)
download_data:list[dict] = Datas.model_dump()
pd.DataFrame(download_data)

#if __name__ == '__main__':
#    main()

下載成功


,站點名稱,行政區,總車輛數,可借,可還,時間
0,YouBike2.0_捷運科技大樓站,大安區,28.0,5.0,23.0,2024-03-29 00:53:18
1,YouBike2.0_復興南路二段273號前,大安區,21.0,5.0,16.0,2024-03-29 00:25:18
2,YouBike2.0_國北教大實小東側門,大安區,16.0,3.0,13.0,2024-03-28 22:45:19
3,YouBike2.0_和平公園東側,大安區,11.0,3.0,8.0,2024-03-29 00:09:18
4,YouBike2.0_辛亥復興路口西北側,大安區,16.0,3.0,13.0,2024-03-29 00:50:26
...,...,...,...,...,...,...
1374,YouBike2.0_臺大總圖書館西南側,臺大公館校區,30.0,3.0,27.0,2024-03-29 00:56:18
1375,YouBike2.0_臺大黑森林西側,臺大公館校區,20.0,3.0,17.0,2024-03-29 00:28:17
1376,YouBike2.0_臺大獸醫館南側,臺大公館校區,24.0,0.0,24.0,2024-03-28 21:33:14
1377,YouBike2.0_臺大新體育館東南側,臺大公館校區,40.0,0.0,40.0,2024-03-28 22:21:14


In [8]:
#宗源學友程式碼
import requests
from requests import Response
import time
import pandas as pd

from pydantic import BaseModel, RootModel, Field

class BikeOfSite(BaseModel):
    行政區:str = Field(alias='sarea')
    站點名稱:str = Field(alias='sna')
    時間:str = Field(alias='mday')
    總車輛數:int = Field(alias='tot')
    可借:int = Field(alias='sbi')
    可還:int = Field(alias='bemp')

############################

class BikeOfSites(RootModel):
    root:list[BikeOfSite]

    def __iter__(self):
        return iter(self.root)
    
    def __getitem__(self, idx):
        return self.root[idx]

############################

def get_response() -> Response | None:
    url_path:str = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'

    count:int = 0
    limit:int = 3
    delay:int = 5

    while True and count < limit:
        try:
            print(f'執行第 {count + 1} 次下載:')
            response:Response = requests.get(url_path, timeout=15)
            response.raise_for_status()
        except Exception as e:
            count += 1
            print('連線失敗。')
            print(e)

            if count < limit:
                print(f'等待 {delay} 秒後再次嘗試。')
                time.sleep(delay)
        else:
            if response.status_code != 200:
                count += 1
                print('下載失敗。')
                print(response.reason)
                
                if count < limit:
                    print(f'等待 {delay} 秒後再次嘗試。')
                    time.sleep(delay)
            else:
                break
    
    if count >= limit:
        print(f'失敗已達 {limit} 次，不再執行下載動作。')
        return None
    else:
        print('下載成功')
        return response

############################

def jsonToList(raw:str) -> list[dict] | None:
    try:
        datas:BikeOfSites = BikeOfSites.model_validate_json(raw)
        items:list[dict] = datas.model_dump()
        return items
    except Exception as e:
        print('預想外的格式。')
        print(e)
        return None

############################
    
#若寫在function裡，例如main()，則pd.DataFrame()不會顯示Table
#沒有報錯，print()顯示有資料，原因不明
# def main():
#     response:Response | None = get_response()

#     if not response:
#         print('dsfghdfh111')
#         return
    
#     rawString:str = response.text
#     download_data:list[dict] | None = jsonToList(rawString)

#     if not download_data:
#         return

#     pd.DataFrame(download_data)
    
# main()

############################

response:Response | None = get_response()

# #寫在if判斷裡，pd.DataFrame()也不會顯示
# if response:
#     rawString:str = response.text
#     download_data:list[dict] | None = jsonToList(rawString)

#     if download_data:
#         pd.DataFrame(download_data)
#         print(len(download_data))

rawString:str = response.text

download_data:list[dict] | None = jsonToList(rawString)
pd.DataFrame(download_data)

執行第 1 次下載:
下載成功


,行政區,站點名稱,時間,總車輛數,可借,可還
0,大安區,YouBike2.0_捷運科技大樓站,2024-03-29 00:21:17,28,7,21
1,大安區,YouBike2.0_復興南路二段273號前,2024-03-29 00:25:18,21,5,16
2,大安區,YouBike2.0_國北教大實小東側門,2024-03-28 22:45:19,16,3,13
3,大安區,YouBike2.0_和平公園東側,2024-03-29 00:09:18,11,3,8
4,大安區,YouBike2.0_辛亥復興路口西北側,2024-03-29 00:18:18,16,2,14
...,...,...,...,...,...,...
1374,臺大公館校區,YouBike2.0_臺大總圖書館西南側,2024-03-29 00:18:18,30,1,29
1375,臺大公館校區,YouBike2.0_臺大黑森林西側,2024-03-29 00:28:17,20,3,17
1376,臺大公館校區,YouBike2.0_臺大獸醫館南側,2024-03-28 21:33:14,24,0,24
1377,臺大公館校區,YouBike2.0_臺大新體育館東南側,2024-03-28 22:21:14,40,0,40
